In [1]:
import pandas as pd 
import geopandas as gpd
import libpysal


In [2]:
ct_nyc = gpd.read_file("../../data/ctwi_nyc/nyct2020wi.shp")

In [3]:
ct_nyc 

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,GEOID,PUMA,Shape_Leng,Shape_Area,geometry
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,4121,11023.048472,1.844421e+06,"MULTIPOLYGON (((972081.788 190733.467, 972184...."
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4103,8010.305397,1.887578e+06,"POLYGON ((988012.660 196152.484, 987984.769 19..."
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,4103,9372.015493,4.429331e+06,"POLYGON ((986961.185 199553.643, 987206.139 19..."
3,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,4103,5075.332033,1.006116e+06,"POLYGON ((987382.076 199473.760, 987206.139 19..."
4,14.02,1,Manhattan,001402,1001402,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4103,4459.156128,1.226206e+06,"POLYGON ((988536.642 200278.524, 988404.814 19..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,176,5,Staten Island,017600,5017600,None,Annadale-Huguenot-Prince's Bay-Woodrow,SI0304,SI03,SI03 South Shore (CD 3 Approximation),36085017600,4503,33259.444647,5.820851e+07,"POLYGON ((938074.526 134571.590, 938187.252 13..."
2323,228.02,5,Staten Island,022802,5022802,None,Freshkills Park (North),SI0291,SI02,SI02 Mid-Island (CD 2 Approximation),36085022802,4502,44266.026098,6.815375e+07,"POLYGON ((938205.274 154530.769, 937703.178 15..."
2324,291.02,5,Staten Island,029102,5029102,None,New Springville-Willowbrook-Bulls Head-Travis,SI0204,SI02,SI02 Mid-Island (CD 2 Approximation),36085029102,4502,55873.335929,1.175749e+08,"POLYGON ((932410.624 168847.921, 932435.091 16..."
2325,161,2,Bronx,016100,2016100,None,Crotona Park East,BX0303,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,36005016100,4263,6476.483410,2.574284e+06,"POLYGON ((1015839.339 245295.043, 1016167.845 ..."


In [4]:
ct_nyc_neighbors = libpysal.weights.Queen.from_dataframe(ct_nyc)

/tmp/ipykernel_2853647/1319026532.py:1: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  ct_nyc_neighbors = libpysal.weights.Queen.from_dataframe(ct_nyc)
/share/ju/conda_virtualenvs/urbanekg/lib/python3.11/site-packages/libpysal/weights/contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
 There is 1 island with id: 0.
  W.__init__(self, neighbors, ids=ids, **kw)


In [5]:
ct_nyc_adj_mtx, ct_nyc_ids = ct_nyc_neighbors.full()

In [6]:
ct_nyc_adj_mtx.shape

(2327, 2327)

In [7]:
ct_nyc_adj_mtx

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [8]:
# format that stan model wants the adjacency matrix in
# two list format, but stan is 1-indexed so offset by 1.
ct_nyc_adj_list = []
for i, row in enumerate(ct_nyc_adj_mtx):
    for j, val in enumerate(row):
        if val == 1:
            ct_nyc_adj_list.append([i+1, j+1])


In [9]:
# write the adjacency list as two separate files, a list of the first column (node1) and a list of the second column (node2)
with open("../../data/processed/ct_nyc_adj_list_node1.txt", "w") as f:
    for item in ct_nyc_adj_list:
        f.write("%s\n" % item[0])
    
with open("../../data/processed/ct_nyc_adj_list_node2.txt", "w") as f:
    for item in ct_nyc_adj_list:
        f.write("%s\n" % item[1])





In [10]:
# write adj matrix and ids as np arrays
import numpy as np
np.save('../../data/processed/ct_nyc_adj_mtx.npy', ct_nyc_adj_mtx)
np.save('../../data/processed/ct_nyc_ids.npy', ct_nyc_ids)

pd.DataFrame(ct_nyc_adj_mtx).to_csv('../../data/processed/ct_nyc_adj_mtx.csv')